In [ ]:
#pip install qiskit==1.0.2 qiskit-aer==0.14.1 qiskit-ibm-runtime==0.20.0
#pip install rustworkx==0.13.1 --no-cache-dir

In [1]:
import numpy as np
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator as AerEstimator

In [ ]:
seed = 20
n_qubits = 2

obs = SparsePauliOp(['ZY', 'YI', 'IY', 'II', 'YY', 'ZY', 'XI', 'YZ', 'ZZ', 'IX'],
              coeffs=[0.64414354+0.j, 0.38074849+0.j, 0.66304791+0.j, 0.16365073+0.j,
 0.96260781+0.j, 0.34666184+0.j, 0.99175099+0.j, 0.2350579 +0.j,
 0.58569427+0.j, 0.4066901 +0.j])

# Ansatz
circuit = TwoLocal(
    n_qubits,
    ["rx", "ry", "rz"],
    ["cx"],#
    "linear",
    reps=3,
)

params = np.random.uniform(low=0, high=2 * np.pi, size=circuit.num_parameters)

# Estimator V2
aer_sim = AerSimulator(method="automatic")
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_qc = pm.run(circuit)

# Aer Estimator
est = AerEstimator(run_options={"seed": seed, "shots":1024})
result = est.run([isa_qc], [obs], [params]).result()
print(f"Expectation value from AerEstimator: {result.values[0]}")

# Primitive Estimator
est = Estimator(options={"seed": seed})
result = est.run([isa_qc], [obs], [params]).result()
print(f"Expectation value from qiskit.primitve Estimator: {result.values[0]}")

Expectation value from AerEstimator: 1.942484661953125
Expectation value from qiskit.primitve Estimator: 2.035857647344927


In [18]:
from susy_qm import calculate_Hamiltonian2
from qiskit.circuit import QuantumCircuit, Parameter

In [6]:
potential = "DW"
shots = 1024
cutoff = 8

    
H = calculate_Hamiltonian2(cutoff, potential)
eigenvalues = np.sort(np.linalg.eigvals(H))[:4]
num_qubits = int(1 + np.log2(cutoff))

obs = SparsePauliOp.from_operator(H)

In [9]:
circuit = TwoLocal(
    num_qubits,
    ["rx", "ry", "rz"],
    ["cx"],#
    "linear",
    reps=3,
)

In [17]:
seed=42
params = np.random.uniform(low=0, high=2 * np.pi, size=circuit.num_parameters)

# Estimator V2
aer_sim = AerSimulator(method="automatic")
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
isa_qc = pm.run(circuit)

# Aer Estimator
est = AerEstimator(run_options={"seed": seed, "shots":1024})
result = est.run([isa_qc], [obs], [params]).result()
print(f"Expectation value from AerEstimator: {result.values[0]}")

Expectation value from AerEstimator: 11.561350647421376


In [19]:
def cost_function(params, H):

    observable = SparsePauliOp.from_operator(H)
    param_objs = [Parameter(f"θ{i}") for i in range(4)]

    qc = QuantumCircuit(num_qubits)
    qc.ry(param_objs[0], 0)     
    qc.ry(param_objs[1], 1)     
    qc.ry(param_objs[2], 2)     
    qc.ry(param_objs[3], 3)

    aer_sim = AerSimulator(method='automatic')
    pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
    isa_qc = pm.run(qc)

    estimator = AerEstimator(run_options={"seed": 42, "shots":shots})
    result = estimator.run([isa_qc], [observable], [params]).result()
    
    energy = result.values[0]

    return energy

In [22]:
params = np.random.rand(4)
cost_function(params,H)

C:\Users\johnkerf\AppData\Local\Temp\ipykernel_8100\1901656835.py:2: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstimator from qiskit.primitives.
  cost_function(params,H)


8.744237043369203